In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem.porter import PorterStemmer
import seaborn as sb
import re
from nltk.corpus import stopwords
import string
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
tsv_file='train.tsv'
csv_file=pd.read_table(tsv_file,sep='\t')
csv_file.to_csv('train.csv',index=False)

In [3]:
train_news=pd.read_csv('train.csv')

In [4]:
tsv_file='test.tsv'
csv_file=pd.read_table(tsv_file,sep='\t')
csv_file.to_csv('test.csv',index=False)
test_news=pd.read_csv('test.csv')

In [5]:
tsv_file='valid.tsv'
csv_file=pd.read_table(tsv_file,sep='\t')
csv_file.to_csv('valid.csv',index=False)
valid_news=pd.read_csv('valid.csv')

In [6]:
train_news.shape

(10239, 14)

In [7]:
test_news.shape

(1266, 14)

In [8]:
valid_news.shape

(1283, 14)

In [9]:
train_news.isnull().sum()

2635.json                                                                                0
false                                                                                    0
Says the Annies List political group supports third-trimester abortions on demand.       0
abortion                                                                                 2
dwayne-bohac                                                                             2
State representative                                                                  2897
Texas                                                                                 2208
republican                                                                               2
0                                                                                        2
1                                                                                        2
0.1                                                                                      2

In [10]:

train_news.columns=['news_id','label','statement','subject','speaker','job_title','state_info','party_affiliation','barely_true_counts','false_counts','half_true_counts','mostly_true_counts','pants_on_fire_counts','context']
test_news.columns=['new_id','label','statement','subject','speaker','job_title','state_info','party_affiliation','barely_true_counts','false_counts','half_true_counts','mostly_true_counts','pants_on_fire_counts','context']

In [11]:
dataset=pd.concat([train_news,test_news],axis=0)
dataset.head(10)
dataset.columns
dataset.shape
dataset.head(10)

,news_id,label,statement,subject,speaker,job_title,state_info,party_affiliation,barely_true_counts,false_counts,half_true_counts,mostly_true_counts,pants_on_fire_counts,context,new_id
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,NaN
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,NaN
2,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,NaN
3,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,NaN
4,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece,NaN
5,2342.json,barely-true,Jim Dunnam has not lived in the district he re...,candidates-biography,republican-party-texas,NaN,Texas,republican,3.0,1.0,1.0,3.0,1.0,a press release.,NaN
6,153.json,half-true,I'm the only person on this stage who has work...,ethics,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,"a Democratic debate in Philadelphia, Pa.",NaN
7,5602.json,half-true,"However, it took $19.5 million in Oregon Lotte...",jobs,oregon-lottery,NaN,NaN,organization,0.0,0.0,1.0,0.0,1.0,a website,NaN
8,9741.json,mostly-true,Says GOP primary opponents Glenn Grothman and ...,"energy,message-machine-2014,voting-record",duey-stroebel,State representative,Wisconsin,republican,0.0,0.0,0.0,1.0,0.0,an online video,NaN
9,7115.json,mostly-true,"For the first time in history, the share of th...",elections,robert-menendez,U.S. Senator,New Jersey,democrat,1.0,3.0,1.0,3.0,0.0,a speech,NaN


In [12]:
dataset.columns

Index(['news_id', 'label', 'statement', 'subject', 'speaker', 'job_title',
       'state_info', 'party_affiliation', 'barely_true_counts', 'false_counts',
       'half_true_counts', 'mostly_true_counts', 'pants_on_fire_counts',
       'context', 'new_id'],
      dtype='object')

In [13]:
dataset_new=dataset.drop(["news_id","subject","speaker","job_title","state_info","party_affiliation","barely_true_counts","false_counts","half_true_counts","mostly_true_counts","pants_on_fire_counts","context","new_id"],axis=1)

In [14]:
dataset_new.isnull().sum

<bound method NDFrame._add_numeric_operations.<locals>.sum of       label  statement
0     False      False
1     False      False
2     False      False
3     False      False
4     False      False
...     ...        ...
1261  False      False
1262  False      False
1263  False      False
1264  False      False
1265  False      False

[11505 rows x 2 columns]>

In [15]:
dataset_new.head()

,label,statement
0,half-true,When did the decline of coal start? It started...
1,mostly-true,"Hillary Clinton agrees with John McCain ""by vo..."
2,false,Health care reform legislation is likely to ma...
3,half-true,The economic turnaround started at the end of ...
4,true,The Chicago Bears have had more starting quart...


In [16]:
ps=PorterStemmer()
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)#remove newline characters and keep them in single line
    text = re.sub('\w*\d\w*', '', text) #removing all words that have numbers in them 
    text=[ps.stem(word) for word in text if not word in stopwords.words('english')]
    text=' '.join(text)
    return text

In [17]:
dataset_new["statement"]=dataset_new["statement"].apply(wordopt)

In [18]:
#x=dataset_new.iloc[:,-1].values
#y=dataset_new.iloc[:,:-1].values

['w h e n     h e   e c l n e   f   c l   r       r e   w h e n   n u r l   g   k   f f   h   r e     b e g n   n     p r e e n   g e r g e   w       b u h   n r n  '
 'h l l r   c l n n   g r e e   w h   j h n   c c n     b   v n g     g v e   g e r g e   b u h   h e   b e n e f   f   h e   u b   n   r n    '
 'h e l h   c r e   r e f r   l e g l n     l k e l     n e   f r e e   e x   c h n g e   u r g e r e  '
 ...
 'n   h e   e r l       e n     e w r   k e n n e   e c r e l   f f e r e     h e l p   v e   l e e r   c u n e r   h e   r e g n   n r n   p n   n   n u c l e r   r e n  '
 '  n   e p   p e r   l n g u h e   u n e r   r c k l n   b u   h   n e w   e p   r e c r   g     n e   n   w    '
 '  h e   g v e r n r     g n g   r u n   h e   e   l k n g   b u     w e   h u l   f u n   n   n c e   x   c u   h   b e n e f   h g h e r   n c e   e r n e r   n   n   l w e r   n c e   e r n e r']


In [19]:
#dataset_new.head()

,label,statement
0,half-true,w h e n h e e c l n e f c l r ...
1,mostly-true,h l l r c l n n g r e e w h j h n c ...
2,false,h e l h c r e r e f r l e g l n l k ...
3,half-true,h e e c n c u r n r u n r e h e e ...
4,true,h e c h c g b e r h v e h r e r n ...


In [27]:
dataset_new.dtypes

label        object
statement    object
dtype: object

In [20]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [30]:
cat_column=['label']
#ct=ColumnTransformer(transformers=[("encoder",OneHotEncoder(),[0])],remainder="passthrough")
ct=ColumnTransformer([('one_hot_encoder',OneHotEncoder(),cat_column)],remainder='passthrough')
data_encoded=ct.fit_transform(cat_column)
col_names=ct.named_transformers_['one_hot_encoder'].get_feature_names(cat_column)
dataset_enc=pd.DataFrame(data_encoded,columns=list(col_names))
print(dataset_enc)

ValueError: Expected 2D array, got 1D array instead:
array=['label'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [22]:
dataset_new.columns

Index(['label', 'statement'], dtype='object')

In [ ]:
col_names=ct.names_transformers_['one_hot_encoder'].get_feature_names